In [68]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")

In [12]:
full_data = pd.read_csv('Full+data.csv')

# Remove some columns in full+data
x = full_data.columns[14:256]
full_data = full_data.drop(x,axis=1)
# full_data = full_data.drop(['created_at','updated_at','deleted_at'],axis=1)
# Clean/organize values in these columns
full_data['product_id'] = full_data['product_id'].str.upper()
full_data['brand'] = full_data['brand'].str.title()
full_data['name'] = full_data['name'].str.title()
full_data['description'] = full_data['description'].str.title()
full_data['brand_category'] = full_data['brand_category'].str.title()
full_data['details'] = full_data['details'].str.title()
full_data['brand_canonical_url'] = full_data['brand_canonical_url'].str.lower()

full_data.shape

(42373, 14)

In [11]:
extra = pd.read_csv('extra_data.csv')
extra.shape

(6621, 14)

In [7]:
attr = pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx', sheet_name = 'Data')
attr['attribute_name'] = attr['attribute_name'].str.lower()
attr['attribute_value'] = attr['attribute_value'].str.lower()

attr.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,women
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,primary color,blacks
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,casual
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,modern
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,medium


In [323]:
labels = attr.pivot_table(values='attribute_value', 
                          index=attr.product_id, 
                          columns='attribute_name', 
                          aggfunc='first',
                          dropna = False).reset_index()
labels = labels.add_prefix('attribute_')
labels

attribute_name,attribute_product_id,attribute_additional color,attribute_calf_width,attribute_category,attribute_class_blazers_coats_and_jackets,attribute_class_booties,attribute_class_boots,attribute_class_dress,attribute_class_flats,attribute_class_handbags,...,attribute_subcategory_bottom,attribute_subcategory_one_piece,attribute_subcategory_shoe,attribute_subcategory_sweater,attribute_subcategory_sweatshirt_and_hoodie,attribute_subcategory_top,attribute_toe_style,attribute_trend,attribute_upper_material,attribute_wash
0,01DMBRYVA2P5H24WK0HTK4R0A1,NaN,NaN,bottom,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01DMBRYVA2PEPWFTT7RMP5AA1T,NaN,NaN,"blazers, coats & jackets",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01DMBRYVA2Q2ST7MNYR6EEY4TK,NaN,NaN,one piece,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01DMBRYVA2S5T9W793F4CY41HE,NaN,NaN,accessory,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01DMBRYVA2ZFDYRYY5TRQZJTBD,NaN,NaN,shoe,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,01E2M4B1MS13817G2YZA3J8ZD6,NaN,NaN,sweater,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN,NaN
922,01E2M4BTQCZMT0WRCAMT9CX3PY,NaN,NaN,bottom,NaN,NaN,NaN,NaN,NaN,NaN,...,pants & leggings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
923,01E2M4E8HZ58197FV7WZ4DX80K,browns,NaN,sweater,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN,NaN
924,01E2M4KKENFX66KPTHA8PRMYC3,pinks,NaN,sweatshirt & hoodie,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN


In [324]:
unique_prod_id = list(set(full_data['product_id'])) + list(set(extra['product_id']))
len(unique_prod_id)

48994

In [325]:
# Select related columns for EDA

data = full_data[['product_id','brand','name','description','brand_category']] #,'details'
print(data.columns)
data.tail()

Index(['product_id', 'brand', 'name', 'description', 'brand_category'], dtype='object')


,product_id,brand,name,description,brand_category
42368,01DT5110V2ME7BY3JPCAJ91QRW,Mara Hoffman,Atlas Oversized Belted Mélange Wool Coat,Mélange Beige And Cream Wool Button Fastenings...,Clothing / Coats / Long
42369,01DV6M2FXMPW9RSZWSXW1EK75W,Philosophy Di Lorenzo Serafini,Cropped Crochet-Trimmed Georgette Top,"Cream Georgette Ties At Neck, Concealed Hook F...",Clothing / Tops / Blouses
42370,01DV71D0XMBM4VVJK54F2HD3ZG,Vanessa Bruno,Juna Cotton-Corduroy Mini Skirt,Sand Cotton-Corduroy Concealed Hook And Zip Fa...,Clothing / Skirts / Mini
42371,01DV72R28K2A1AN5G167S6QRWF,Eve Denim,Annabel Rigid Mid-Rise Skinny Jean,Although Mom Jeans And Boyfriend Jeans Are All...,Women:Clothing:Jeans
42372,01DSNZTZNH0NG60JWJ07KPQ465,Fila,Kessler Colorblock Track Jacket,Only At Saks. Ultra-Sporty Nylon Track Jacket ...,Themensstore/Apparel/Outerwear/Lightweightjackets


In [326]:
# Data clearning
data['brand_category'] = data['brand_category'].str.replace(r'(/|,|:)',' ')
data['brand_category'] = data['brand_category'].str.lower()
data['brand'] = data['brand'].str.title()
data['name'] = data['name'].str.title()
data['description'] = data['name'].str.title()

data.tail()

,product_id,brand,name,description,brand_category
42368,01DT5110V2ME7BY3JPCAJ91QRW,Mara Hoffman,Atlas Oversized Belted Mélange Wool Coat,Atlas Oversized Belted Mélange Wool Coat,clothing coats long
42369,01DV6M2FXMPW9RSZWSXW1EK75W,Philosophy Di Lorenzo Serafini,Cropped Crochet-Trimmed Georgette Top,Cropped Crochet-Trimmed Georgette Top,clothing tops blouses
42370,01DV71D0XMBM4VVJK54F2HD3ZG,Vanessa Bruno,Juna Cotton-Corduroy Mini Skirt,Juna Cotton-Corduroy Mini Skirt,clothing skirts mini
42371,01DV72R28K2A1AN5G167S6QRWF,Eve Denim,Annabel Rigid Mid-Rise Skinny Jean,Annabel Rigid Mid-Rise Skinny Jean,women clothing jeans
42372,01DSNZTZNH0NG60JWJ07KPQ465,Fila,Kessler Colorblock Track Jacket,Kessler Colorblock Track Jacket,themensstore apparel outerwear lightweightjackets


In [372]:
input_data = data
result_df1 = pd.DataFrame(input_data['product_id'])
result_df1['brand'] = input_data['brand']
result_df1['combined_data'] = data.drop(labels = ['brand','product_id'], axis = 1).apply(' '.join, axis=1)



# function remove stopword
# combine all columns - call combin_col() function
data2 = combine_col(input_data)

# remove_stopwords
from nltk.corpus import stopwords
import re

nltk_stopwords = list((set(stopwords.words('english'))))
# remove "unknown" since in the full_data dataset, some brand_category are unknown
nltk_stopwords.append('unknown')


regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")

result2 = []

for line in data2['combined_data']:
    filtered_words = []
    if isinstance(line, str):
        line = re.sub(r'\d+\+*[\- ]*[\-]*',' ',line)
        for word in regex_word_tokenize.tokenize(line):
            if word.isdigit() == False:
                if word.lower() not in nltk_stopwords:
                    filtered_words.append(word.lower())
        result2.append(" ".join(filtered_words))
    else:
        result2.append(np.nan)
result_df2 = result_df1[['product_id','brand']]
result_df2['result'] = result2
# result_df2['result'] = result_df2['result'] + " " + input_data['brand']




# funtion lemmatize
d1 = result_df2
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk_stopwords = list((set(stopwords.words('english'))))
lemmatizer = WordNetLemmatizer()

result3 = []
for line in d1['result']:
    lemmatized = []
    for word in regex_word_tokenize.tokenize(line):
        lemmatized.append(lemmatizer.lemmatize(word))
    result3.append(" ".join(lemmatized))
result_df3 = result_df2[['product_id','brand']]
result_df3['result'] = result3
result_df3['result'] = result_df3['result'] + " " + result_df2['brand']

In [366]:
result_df3.result[0]

'original fitness sneaker original fitness sneaker themensstore shoe sneaker lowtop Fila'

In [367]:
result_df2.result[0]

'original fitness sneakers original fitness sneakers themensstore shoes sneakers lowtop'

In [368]:
# Function:
# Combine all inputs/columns into one string

def combine_col(input_data):
    '''Combine all inputs/columns into one string.
       Returns a DataFrame'''
    result_df1 = pd.DataFrame(input_data['product_id'])
    result_df1['brand'] = input_data['brand']
    result_df1['combined_data'] = data.drop(labels = ['brand','product_id'], axis = 1).apply(' '.join, axis=1)
    return result_df1


In [373]:
# Function:
# Tokenize string and remove stopwords

def remove_stopwords(input_data):
    '''Tokenize string.
        Returns a list.'''
    
    # combine all columns - call combin_col() function
    data2 = combine_col(input_data)

    # remove_stopwords
    from nltk.corpus import stopwords
    import re

    nltk_stopwords = list((set(stopwords.words('english'))))
    # remove "unknown" since in the full_data dataset, some brand_category are unknown
    nltk_stopwords.append('unknown')


    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")

    result2 = []

    for line in data2['combined_data']:
        filtered_words = []
        if isinstance(line, str):
            line = re.sub(r'\d+\+*[\- ]*[\-]*',' ',line)
            for word in regex_word_tokenize.tokenize(line):
                if word.isdigit() == False:
                    if word.lower() not in nltk_stopwords:
                        filtered_words.append(word.lower())
            result2.append(" ".join(filtered_words))
        else:
            result2.append(np.nan)
    result_df2 = result_df1[['product_id','brand']]
    result_df2['result'] = result2
    return result_df2


In [370]:
def lemmatize_word(input_data):
    d1 = remove_stopwords(input_data)
    
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    nltk_stopwords = list((set(stopwords.words('english'))))
    lemmatizer = WordNetLemmatizer()
    
    result3 = []
    for line in d1['result']:
        lemmatized = []
        for word in regex_word_tokenize.tokenize(line):
            lemmatized.append(lemmatizer.lemmatize(word))
        result3.append(" ".join(lemmatized))
    result_df3 = result_df2[['product_id','brand']]
    result_df3['result'] = result3
    result_df3['result'] = result_df3['result'] + " " + result_df2['brand']
    return result_df3.drop(labels = ['brand'], axis = 1)

In [379]:
# N-Gram

input_data = lemmatize_word(data)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(2,2), min_df = 2)
gram = vectorizer.fit_transform(input_data["result"])
gram_features = pd.DataFrame(gram.toarray(), columns = vectorizer.get_feature_names())
print(f'Shape of X_pos: {gram_features.shape}')


from sklearn.feature_extraction.text import TfidfVectorizer

# Kept the number of features at 5000 because my computer cannot handle more than 5000 (the kernel will die)
tfidfvectorizer = TfidfVectorizer(ngram_range = (2,2),
#                                   token_pattern = r"\w+['-]?[a-zA-Z]*",
                                  min_df = 2)
corpus = list(input_data["result"].values)
X = tfidfvectorizer.fit_transform(corpus)
X_terms = tfidfvectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=X_terms)
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)
score

# gram_features

Shape of X_pos: (42373, 30155)


,score
themensstore apparel,767.633331
sole society,737.167282
themensstore accessory,522.495562
banana republic,424.568533
justkids girl,420.356001
...,...
set ox,0.326429
spf anti,0.322962
theitlistq sofreshsoclean,0.305638
acnetreatments trish,0.289465


In [380]:
pd.merge(left = input_data, right = attr, left_on = 'product_id', right_on = 'product_id')

,product_id,result,product_color_id,attribute_name,attribute_value
0,01DTJCERF6F4NRZ2WSJFFA1EYS,teah stretch-silk camisole teah stretch-silk c...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,neckline,scoop neck
1,01DTJCERF6F4NRZ2WSJFFA1EYS,teah stretch-silk camisole teah stretch-silk c...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,sizing,regular
2,01DTJCERF6F4NRZ2WSJFFA1EYS,teah stretch-silk camisole teah stretch-silk c...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,weekend
3,01DTJCERF6F4NRZ2WSJFFA1EYS,teah stretch-silk camisole teah stretch-silk c...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,work
4,01DTJCERF6F4NRZ2WSJFFA1EYS,teah stretch-silk camisole teah stretch-silk c...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,length_top,mid-length (at hips)
...,...,...,...,...,...
18026,01DV72R28K2A1AN5G167S6QRWF,annabel rigid mid-rise skinny jean annabel rig...,01DV72R2BDQ4GKVVZ1YZK5FKKF,style,classic
18027,01DV72R28K2A1AN5G167S6QRWF,annabel rigid mid-rise skinny jean annabel rig...,01DV72R2BDQ4GKVVZ1YZK5FKKF,wash,medium blue
18028,01DV72R28K2A1AN5G167S6QRWF,annabel rigid mid-rise skinny jean annabel rig...,01DV72R2BDQ4GKVVZ1YZK5FKKF,material_clothing,denim
18029,01DV72R28K2A1AN5G167S6QRWF,annabel rigid mid-rise skinny jean annabel rig...,01DV72R2BDQ4GKVVZ1YZK5FKKF,class_pants_and_leggings,denim (jeans)
